In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re
import html
from bs4 import BeautifulSoup
import ftfy
import pandas as pd

In [5]:
dockets = ["TTB-2025-0003", "TTB-2025-0002"]

In [6]:
try:
    REPO_ROOT = Path(__file__).parent.parent.resolve()
except NameError:
    REPO_ROOT = Path(os.getcwd()).parent.resolve()

OUTPUTS_DIR = REPO_ROOT / "outputs"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
comments_paths = []
summary_paths = []

for i in dockets:
    comments_path = OUTPUTS_DIR / f"comments_with_bertopic_{i}.csv"
    summary_path    = OUTPUTS_DIR / f"bertopic_topic_summary_{i}.csv"
    comments_paths.append(comments_path)
    summary_paths.append(summary_path)

In [8]:
print(comments_paths)

[WindowsPath('C:/Users/linna/OneDrive/Documents/Python_Dev/topic-modeling/outputs/comments_with_bertopic_TTB-2025-0003.csv'), WindowsPath('C:/Users/linna/OneDrive/Documents/Python_Dev/topic-modeling/outputs/comments_with_bertopic_TTB-2025-0002.csv')]


In [9]:
comments_dfs = []
for p in comments_paths:
    if not p.exists():
        print(f"Warning: missing file {p}; skipping.")
        continue
    comments_dfs.append(pd.read_csv(p))

In [10]:
comments_dfs = []

for p in comments_paths:
    if not p.exists():
        print(f"Warning: missing file {p}; skipping.")
        continue
    comments_dfs.append(pd.read_csv(p))

In [11]:
summary_dfs = []

for p, docket_id in zip(summary_paths, dockets):
    if not p.exists():
        print(f"Warning: missing file {p}; skipping.")
        continue
    df = pd.read_csv(p)
    df["docket_id"] = docket_id
    summary_dfs.append(df)

In [12]:
if comments_dfs:
    all_comments = pd.concat(comments_dfs, ignore_index=True)
else:
    all_comments = pd.DataFrame()

In [13]:
if summary_dfs:
    all_summary = pd.concat(summary_dfs, ignore_index=True)
else:
    all_summary = pd.DataFrame()

### Basic cleaning to remove characters like \<br/>

In [15]:
def clean_comment_text(s: str) -> str:
    if pd.isna(s):
        return s

    soup_text = BeautifulSoup(s, "html.parser").get_text(" ")
    
    unescaped = html.unescape(soup_text)
    
    fixed = ftfy.fix_text(unescaped)
    fixed = re.sub(r'\b(?:mdash|ndash|nbsp|nbsp;|amp|lt|gt|quot|rsquo|lsquo|ldquo|rsquo;|rdquo|br|br/)\b',
                   ' ', fixed, flags=re.IGNORECASE)
    
    fixed = re.sub(r'\d+\s*px', ' ', fixed, flags=re.IGNORECASE)   # remove 30px, 12 px etc
    fixed = re.sub(r'\b(?:span|div|padding-left|class|style|font-family|font-size)\b', ' ', fixed, flags=re.IGNORECASE)
    fixed = re.sub(r'\s+', ' ', fixed).strip()

    return fixed

In [16]:
all_comments['comment_text'] = all_comments['comment_text'].map(clean_comment_text)

C:\Users\linna\AppData\Local\Temp\ipykernel_2584\1140927365.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_text = BeautifulSoup(s, "html.parser").get_text(" ")


In [17]:
all_summary['sample_comments'] = all_summary['sample_comments'].map(clean_comment_text)

In [18]:
# save
all_comments.to_csv(OUTPUTS_DIR / "all_comments_with_bertopic_combined.csv", index=False)
all_summary.to_csv(OUTPUTS_DIR / "all_bertopic_topic_summary_combined.csv", index=False)

In [19]:
# all_comments.to_csv(OUTPUTS_DIR / "all_comments_with_bertopic_combined.csv", index=False)
# all_summary.to_csv(OUTPUTS_DIR / "all_bertopic_topic_summary_combined.csv", index=False)

In [20]:
# -*- coding: utf-8 -*-
from pathlib import Path
import streamlit as st
import pandas as pd
import plotly.express as px
from typing import Optional
import streamlit.components.v1 as components

# --- CONFIG: CSV paths using relative paths ---
# Get the project root (parent of app directory)
PROJECT_ROOT = Path(__file__).parent.parent
DEFAULT_TOPIC_SUMMARY = PROJECT_ROOT / "outputs" / "all_bertopic_topic_summary_combined.csv"
DEFAULT_COMMENTS_DF = PROJECT_ROOT / "outputs" / "all_comments_with_bertopic_combined.csv" 

# Basic page config
st.set_page_config(page_title="Topic Explorer", layout="wide", initial_sidebar_state="expanded")

@st.cache_data(show_spinner=False)
def load_csv_fallback(path: Path) -> Optional[pd.DataFrame]:
    if not path.exists():
        return None
    for enc in ("utf-8", "cp1252", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    return None

topic_summary = load_csv_fallback(DEFAULT_TOPIC_SUMMARY)
comments_df = load_csv_fallback(DEFAULT_COMMENTS_DF)

if topic_summary is None or comments_df is None:
    st.warning("Topic summary or comments CSV not found. Put the files in the expected paths and reload.")
    st.info(f"Expected: topic_summary={DEFAULT_TOPIC_SUMMARY}, comments={DEFAULT_COMMENTS_DF}")
    st.stop()

NameError: name '__file__' is not defined